In [4]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
import statistics
import random
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss
import scipy.misc
# from scipy.misc import imread, imresize


In [5]:
import pickle
import datetime
import pandas as pd
import keras

from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, \
                                       ZeroPadding2D

# from keras.layers.normalization import BatchNormalization
# from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import model_from_json
# from sklearn.metrics import log_loss
from numpy.random import permutation
import time

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/deep_learning_project/

/content/drive/.shortcut-targets-by-id/15gKjVs5Brz2wVAZdIJoqScr7ejFRXBo9/deep_learning_project


In [ ]:
img_rows, img_cols = 64, 64
batch_size = 32
nb_epoch = 1
random_state = 51

In [ ]:
use_cache = 1
# color type: 1 - grey, 3 - rgb
color_type_global = 1
# color_type = 1 - gray
# color_type = 3 - RGB
def get_im_cv2(path, img_rows, img_cols, color_type=1):
    # Load as grayscale
    if color_type == 1:
        img = cv2.imread(path, 0)
    elif color_type == 3:
        img = cv2.imread(path)
    # Reduce size
    resized = cv2.resize(img, (img_cols, img_rows))
    return resized


def get_im_cv2_mod(path, img_rows, img_cols, color_type=1):
    # Load as grayscale
    if color_type == 1:
        img = cv2.imread(path, 0)
    else:
        img = cv2.imread(path)
    # Reduce size
    rotate = random.uniform(-10, 10)
    M = cv2.getRotationMatrix2D((img.shape[1]/2, img.shape[0]/2), rotate, 1)
    img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    resized = cv2.resize(img, (img_cols, img_rows), cv2.INTER_LINEAR)
    return resized
    

def get_driver_data():
    dr = dict()
    # path = os.path.join('..', 'input', 'driver_imgs_list.csv')
    path='/content/drive/MyDrive/deep_learning_project/input/driver_imgs_list.csv'
    print('Read drivers data')
    f = open(path, 'r')
    line = f.readline()
    while (1):
        line = f.readline()
        if line == '':
            break
        arr = line.strip().split(',')
        dr[arr[2]] = arr[0]
    f.close()
    return dr


def load_train(img_rows, img_cols, color_type=1):
    X_train = []
    y_train = []
    driver_id = []
    start_time = time.time()
    driver_data = get_driver_data()



    print('Read train images')
    for j in range(10):
        print('Load folder c{}'.format(j))
        path='/content/drive/MyDrive/deep_learning_project/input/train/c'+str(j)+'/*.jpg'
        files = glob.glob(path)
        i=0
        for fl in files:
          if '(' not in fl:
            flbase = os.path.basename(fl)
            img = get_im_cv2_mod(fl, img_rows, img_cols, color_type=1)
            print('Loading image{}'.format(i))
            i = i+1
            X_train.append(img)
            y_train.append(j)
            driver_id.append(driver_data[flbase])

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    unique_drivers = sorted(list(set(driver_id)))
    print('Unique drivers: {}'.format(len(unique_drivers)))
    print(unique_drivers)
    return X_train, y_train, driver_id, unique_drivers


def load_test(img_rows, img_cols, color_type=1):
    print('Read test images')
    start_time = time.time()
    # path = os.path.join('..', 'input', 'test', '*.jpg')
    path='/content/drive/MyDrive/deep_learning_project/input/test/*.jpg'
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    i = 0
    for fl in files:
      if '(' not in fl:
        flbase = os.path.basename(fl)
        img = get_im_cv2_mod(fl, img_rows, img_cols, color_type)
        X_test.append(img)
        X_test_id.append(flbase)
        print('Loading image{}'.format(i))
        i = i+1
        total += 1
        if total%thr == 0:
            print('Read {} images from {}'.format(total, len(files)))
    
    print('Read test data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_test, X_test_id


def cache_data(data, path):
    if os.path.isdir(os.path.dirname(path)):
        file = open(path, 'wb')
        pickle.dump(data, file)
        file.close()
    else:
        print('Directory doesnt exists')


def restore_data(path):
    data = dict()
    if os.path.isfile(path):
        file = open(path, 'rb')
        data = pickle.load(file)
    return data


def save_model(model):
    json_string = model.to_json()
    if not os.path.isdir('cache'):
        os.mkdir('cache')
    open(os.path.join('cache', 'architecture.json'), 'w').write(json_string)
    model.save_weights(os.path.join('cache', 'model_weights.h5'), overwrite=True)


def read_model():
    model = model_from_json(open(os.path.join('cache', 'architecture.json')).read())
    model.load_weights(os.path.join('cache', 'model_weights.h5'))
    return model


def split_validation_set(train, target, test_size):
    random_state = 51
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test


def create_submission(predictions, test_id, info):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = info + '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)


def read_and_normalize_train_data(img_rows, img_cols, color_type=1):
    cache_path = os.path.join('cache', 'train_r_' + str(img_rows) + '_c_' + str(img_cols) + '_t_' + str(color_type) + '.dat')
    if not os.path.isfile(cache_path) or use_cache == 0:
        train_data, train_target, driver_id, unique_drivers = load_train(img_rows, img_cols, color_type)
        cache_data((train_data, train_target, driver_id, unique_drivers), cache_path)
    else:
        print('Restore train from cache!')
        (train_data, train_target, driver_id, unique_drivers) = restore_data(cache_path)

    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)
    train_data = train_data.reshape(train_data.shape[0], color_type, img_rows, img_cols)
    train_target = np_utils.to_categorical(train_target, 10)
    train_data = train_data.astype('float32')
    train_data /= 255
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, driver_id, unique_drivers


def read_and_normalize_test_data(img_rows, img_cols, color_type=1):
    cache_path = os.path.join('cache', 'test_r_' + str(img_rows) + '_c_' + str(img_cols) + '_t_' + str(color_type) + '.dat')
    if not os.path.isfile(cache_path) or use_cache == 0:
        test_data, test_id = load_test(img_rows, img_cols, color_type)
        cache_data((test_data, test_id), cache_path)
    else:
        print('Restore test from cache!')
        (test_data, test_id) = restore_data(cache_path)

    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.reshape(test_data.shape[0], color_type, img_rows, img_cols)
    test_data = test_data.astype('float32')
    test_data /= 255
    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    return test_data, test_id


def dict_to_list(d):
    ret = []
    for i in d.items():
        ret.append(i[1])
    return ret


def merge_several_folds_mean(data, nfolds):
    a = np.array(data[0])
    for i in range(1, nfolds):
        a += np.array(data[i])
    a /= nfolds
    return a.tolist()


def merge_several_folds_geom(data, nfolds):
    a = np.array(data[0])
    for i in range(1, nfolds):
        a *= np.array(data[i])
    a = np.power(a, 1/nfolds)
    return a.tolist()


def copy_selected_drivers(train_data, train_target, driver_id, driver_list):
    data = []
    target = []
    index = []
    for i in range(len(driver_id)):
        if driver_id[i] in driver_list:
            data.append(train_data[i])
            target.append(train_target[i])
            index.append(i)
    data = np.array(data, dtype=np.float32)
    target = np.array(target, dtype=np.float32)
    index = np.array(index, dtype=np.uint32)
    return data, target, index

In [ ]:
train_data, train_target, driver_id, unique_drivers = read_and_normalize_train_data(img_rows, img_cols, color_type_global)

Streaming output truncated to the last 5000 lines.
Loading image1050
Loading image1051
Loading image1052
Loading image1053
Loading image1054
Loading image1055
Loading image1056
Loading image1057
Loading image1058
Loading image1059
Loading image1060
Loading image1061
Loading image1062
Loading image1063
Loading image1064
Loading image1065
Loading image1066
Loading image1067
Loading image1068
Loading image1069
Loading image1070
Loading image1071
Loading image1072
Loading image1073
Loading image1074
Loading image1075
Loading image1076
Loading image1077
Loading image1078
Loading image1079
Loading image1080
Loading image1081
Loading image1082
Loading image1083
Loading image1084
Loading image1085
Loading image1086
Loading image1087
Loading image1088
Loading image1089
Loading image1090
Loading image1091
Loading image1092
Loading image1093
Loading image1094
Loading image1095
Loading image1096
Loading image1097
Loading image1098
Loading image1099
Loading image1100
Loading image1101
Loading image

In [ ]:
np.save('train_data.npy', train_data)
np.save('train_target.npy', train_target)
np.save('driver_id.npy',driver_id)
np.save('unique_drivers.npy',unique_drivers)


In [ ]:
train_data = np.load('train_data.npy')
train_target = np.load('train_target.npy')
driver_id = np.load('driver_id.npy')
unique_drivers = np.load('unique_drivers.npy')

In [ ]:
test_data, test_id = read_and_normalize_test_data(img_rows, img_cols, color_type_global)

Read test images
Loading image0
Loading image1
Loading image2
Loading image3
Loading image4
Loading image5
Loading image6
Loading image7
Loading image8
Loading image9
Loading image10
Loading image11
Loading image12
Loading image13
Loading image14
Loading image15
Loading image16
Loading image17
Loading image18
Loading image19
Loading image20
Loading image21
Loading image22
Loading image23
Loading image24
Loading image25
Loading image26
Loading image27
Loading image28
Loading image29
Loading image30
Loading image31
Loading image32
Loading image33
Loading image34
Loading image35
Loading image36
Loading image37
Loading image38
Loading image39
Loading image40
Loading image41
Loading image42
Loading image43
Loading image44
Loading image45
Loading image46
Loading image47
Loading image48
Loading image49
Loading image50
Loading image51
Loading image52
Loading image53
Loading image54
Loading image55
Loading image56
Loading image57
Loading image58
Loading image59
Loading image60
Loading image61
L

In [ ]:
np.save('test_data.npy', test_data)
np.save('test_id.npy', test_id)

In [ ]:
test_data = np.load('test_data_sub.npy')
test_id = np.load('test_id_sub.npy')

In [ ]:
def create_model_v1(img_rows, img_cols, color_type=1):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, padding='same', kernel_initializer='he_normal',
                            input_shape=(color_type, img_rows, img_cols)))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(64, 3, 3, padding='same', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(MaxPooling2D((8, 8),padding='same'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(Adam(lr=0.001), loss='categorical_crossentropy',metrics=['accuracy'])
    return model
    
def run_single():
    # input image dimensions
    img_rows, img_cols = 64, 64
    batch_size = 32
    nb_epoch = 100


    yfull_train = dict()
    yfull_test = []

    unique_list_train = ['p002', 'p012', 'p014', 'p016', 'p021', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p051', 'p052', 'p061', 'p064', 'p066', 'p072',
                     'p075','p081']
    X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
    unique_list_valid = ['p050','p015','p022','p056']
    X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

    print('Start Single Run')
    print('Split train: ', len(X_train), len(Y_train))
    print('Split valid: ', len(X_valid), len(Y_valid))
    print('Train drivers: ', unique_list_train)
    print('Test drivers: ', unique_list_valid)
    
    model = create_model_v1(img_rows, img_cols, color_type_global)
    callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)
    his = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch
                    ,verbose=1, validation_data=(X_valid, Y_valid),callbacks=[callback])
    
    return his




his = run_single()

Start Single Run
Split train:  18732 18732
Split valid:  3692 3692
Train drivers:  ['p002', 'p012', 'p014', 'p016', 'p021', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p051', 'p052', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
Test drivers:  ['p050', 'p015', 'p022', 'p056']
Epoch 1/100
586/586 [==============================] - 4s 5ms/step - loss: 2.3042 - accuracy: 0.1363 - val_loss: 2.2639 - val_accuracy: 0.1631
Epoch 2/100
586/586 [==============================] - 3s 5ms/step - loss: 1.9272 - accuracy: 0.2989 - val_loss: 2.3623 - val_accuracy: 0.1896
Epoch 3/100
586/586 [==============================] - 3s 5ms/step - loss: 1.7376 - accuracy: 0.3669 - val_loss: 2.3039 - val_accuracy: 0.2511
Epoch 4/100
586/586 [==============================] - 3s 5ms/step - loss: 1.6173 - accuracy: 0.4187 - val_loss: 2.4949 - val_accuracy: 0.1953
Epoch 5/100
586/586 [==============================] - 3s 5ms/step - loss: 1.5517 - accuracy: 0.4394 - val_loss: 2.3556 

In [ ]:
create_model_v1 = pd.DataFrame(his.history)
create_model_v1.to_csv('create_model_v1.csv')

In [ ]:
def vgg_bn(img_rows, img_cols, color_type=1):
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), padding='same', kernel_initializer='he_normal', input_shape=(color_type, img_rows, img_cols)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(32, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), padding='same',strides=(2, 2),kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(128, (3, 3), padding='same',strides=(2, 2),kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='sigmoid', kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax', kernel_initializer='he_normal'))
    model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
img_rows, img_cols = 64, 64
batch_size = 32
nb_epoch = 100


yfull_train = dict()
yfull_test = []
unique_list_train = ['p002', 'p012', 'p014', 'p016', 'p021', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p051', 'p052', 'p061', 'p064', 'p066', 'p072',
                     'p075','p081']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p050','p015','p022','p056']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)

model = vgg_bn(img_rows, img_cols, color_type_global)
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)
hist = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch
                 ,verbose=1, validation_data=(X_valid, Y_valid),callbacks=[callback])

Start Single Run
Split train:  18732 18732
Split valid:  3692 3692
Train drivers:  ['p002', 'p012', 'p014', 'p016', 'p021', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p051', 'p052', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
Test drivers:  ['p050', 'p015', 'p022', 'p056']
Epoch 1/100
586/586 [==============================] - 6s 8ms/step - loss: 1.6738 - accuracy: 0.4076 - val_loss: 2.6448 - val_accuracy: 0.2793
Epoch 2/100
586/586 [==============================] - 4s 7ms/step - loss: 0.4518 - accuracy: 0.8644 - val_loss: 2.5857 - val_accuracy: 0.3237
Epoch 3/100
586/586 [==============================] - 4s 7ms/step - loss: 0.2506 - accuracy: 0.9270 - val_loss: 2.7102 - val_accuracy: 0.3459
Epoch 4/100
586/586 [==============================] - 4s 7ms/step - loss: 0.1694 - accuracy: 0.9532 - val_loss: 2.9252 - val_accuracy: 0.3191
Epoch 5/100
586/586 [==============================] - 4s 7ms/step - loss: 0.1385 - accuracy: 0.9620 - val_loss: 3.7391 

In [ ]:
vgg_bn = pd.DataFrame(hist.history)
vgg_bn.to_csv('vgg_bn.csv')

# Data augment before vgg_bn

In [ ]:
def vgg_bn(img_rows, img_cols, color_type=1):
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), padding='same', kernel_initializer='he_normal', input_shape=(color_type, img_rows, img_cols)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(32, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), padding='same',strides=(2, 2),kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(128, (3, 3), padding='same',strides=(2, 2),kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='sigmoid', kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax', kernel_initializer='he_normal'))
    model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping
import time

#Record time
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()
#Early stop
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)

#Learning rate schedule



#data augmentation
datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, Y_train, batch_size = 64)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (18732, 1, 64, 64) (64 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


In [ ]:
img_rows, img_cols = 64, 64
batch_size = 64
nb_epoch = 100


yfull_train = dict()
yfull_test = []
unique_list_train = ['p002', 'p012', 'p014', 'p016', 'p021', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p051', 'p052', 'p061', 'p064', 'p066', 'p072',
                     'p075','p081']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p050','p015','p022','p056']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)

model = vgg_bn(img_rows, img_cols, 1)
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)
hist = model.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64, epochs=nb_epoch
                 ,verbose=1, validation_data=(X_valid, Y_valid),callbacks=[callback,time_callback])

Start Single Run
Split train:  18732 18732
Split valid:  3692 3692
Train drivers:  ['p002', 'p012', 'p014', 'p016', 'p021', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p051', 'p052', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
Test drivers:  ['p050', 'p015', 'p022', 'p056']


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
292/292 [==============================] - 51s 170ms/step - loss: 2.2719 - accuracy: 0.1614 - val_loss: 2.7994 - val_accuracy: 0.1446
Epoch 2/100
292/292 [==============================] - 49s 169ms/step - loss: 1.6517 - accuracy: 0.3775 - val_loss: 2.9988 - val_accuracy: 0.2183
Epoch 3/100
292/292 [==============================] - 49s 169ms/step - loss: 1.3976 - accuracy: 0.4722 - val_loss: 2.5813 - val_accuracy: 0.2224
Epoch 4/100
292/292 [==============================] - 49s 169ms/step - loss: 1.2109 - accuracy: 0.5611 - val_loss: 2.5914 - val_accuracy: 0.2470
Epoch 5/100
292/292 [==============================] - 49s 169ms/step - loss: 1.0543 - accuracy: 0.6341 - val_loss: 2.8665 - val_accuracy: 0.2557
Epoch 6/100
292/292 [==============================] - 50s 169ms/step - loss: 0.9511 - accuracy: 0.6716 - val_loss: 2.7273 - val_accuracy: 0.2589
Epoch 7/100
292/292 [==============================] - 50s 169ms/step - loss: 0.8563 - accuracy: 0.7135 - val_loss: 2.9760 -

In [ ]:
def cnn(img_rows, img_cols, color_type=1):
  model = Sequential()

  model.add(Convolution2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(color_type, img_rows, img_cols), kernel_initializer='glorot_normal'))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  model.add(Convolution2D(filters=128, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  model.add(Convolution2D(filters=256, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  model.add(Convolution2D(filters=512, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(500, activation='relu', kernel_initializer='glorot_normal'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))
  model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
from sklearn.model_selection import train_test_split
img_rows, img_cols = 64, 64
batch_size = 32
nb_epoch = 100



unique_list_train = ['p002', 'p012', 'p014', 'p016', 'p021', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p051', 'p052', 'p061', 'p064', 'p066', 'p072',
                     'p075','p081']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p050','p015','p022','p056']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)

model = cnn(img_rows, img_cols, color_type_global)
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)
his = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch
                ,verbose=1, validation_data=(X_valid, Y_valid),callbacks=[callback])

Start Single Run
Split train:  18732 18732
Split valid:  3692 3692
Train drivers:  ['p002', 'p012', 'p014', 'p016', 'p021', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p051', 'p052', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
Test drivers:  ['p050', 'p015', 'p022', 'p056']
Epoch 1/100
586/586 [==============================] - 5s 7ms/step - loss: 1.9273 - accuracy: 0.2667 - val_loss: 2.7315 - val_accuracy: 0.2397
Epoch 2/100
586/586 [==============================] - 4s 6ms/step - loss: 1.0399 - accuracy: 0.6185 - val_loss: 3.0620 - val_accuracy: 0.2638
Epoch 3/100
586/586 [==============================] - 4s 6ms/step - loss: 0.5880 - accuracy: 0.7959 - val_loss: 3.2944 - val_accuracy: 0.3150
Epoch 4/100
586/586 [==============================] - 4s 6ms/step - loss: 0.3826 - accuracy: 0.8788 - val_loss: 3.2624 - val_accuracy: 0.3402
Epoch 5/100
586/586 [==============================] - 4s 6ms/step - loss: 0.2807 - accuracy: 0.9095 - val_loss: 3.3131 

In [ ]:
cnn = pd.DataFrame(his.history)
cnn.to_csv('cnn.csv')

# ResNet50

In [6]:
X_train=np.load('/content/drive/MyDrive/deep_learning_project/X_train.npy')
X_test=np.load('/content/drive/MyDrive/deep_learning_project/X_test.npy')
y_train=np.load('/content/drive/MyDrive/deep_learning_project/y_train.npy')
y_test=np.load('/content/drive/MyDrive/deep_learning_project/y_test.npy')

In [7]:
import tensorflow.keras as K
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf

model_input_shape = (224,224,3)
base_model  =K.applications.resnet.ResNet50(include_top = False,
                                                  weights = 'imagenet',
                                                  input_shape = model_input_shape)
base_model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________

In [8]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)

x = base_model.output
x = K.layers.Flatten()(x)
x = K.layers.Dropout(0.5)(x)

output =K.layers.Dense(units = 10,activation = tf.nn.softmax)(x)
model = K.models.Model(inputs=base_model.inputs, outputs=output)

model.compile(optimizer=K.optimizers.Adam(0.0001),
              loss=K.losses.CategoricalCrossentropy(from_logits = False),
              metrics=['accuracy'])

In [ ]:
hist = model.fit(
    x = X_train,y=y_train,
    validation_data=(X_test,y_test),
    batch_size = 64,
    epochs=100,
    callbacks = [callback,time_callback],
    verbose=1)

Epoch 1/100
270/270 [==============================] - 58s 198ms/step - loss: 0.7466 - accuracy: 0.8483 - val_loss: 1.0245 - val_accuracy: 0.7701
Epoch 2/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0140 - accuracy: 0.9955 - val_loss: 0.8076 - val_accuracy: 0.8080
Epoch 3/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.7510 - val_accuracy: 0.8235
Epoch 4/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0121 - accuracy: 0.9967 - val_loss: 1.7037 - val_accuracy: 0.7748
Epoch 5/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0278 - accuracy: 0.9941 - val_loss: 0.7766 - val_accuracy: 0.8523
Epoch 6/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0172 - accuracy: 0.9967 - val_loss: 0.6524 - val_accuracy: 0.8570
Epoch 7/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0058 - accuracy: 0.9983 - val_loss: 0.9559 -

In [ ]:
res_50_no_augment = pd.DataFrame(hist.history)
res_50_no_augment['time'] = time_callback.times
res_50_no_augment.to_csv('res_50_no_augment.csv')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping
import time



#data augmentation
datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = 64)

# Fits the model on batches with real-time data augmentation:
hist = model.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64, callbacks=[time_callback],
                                                   epochs = 50, verbose = 1, validation_data = (X_test, y_test))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
269/269 [==============================] - 215s 783ms/step - loss: 1.5817 - accuracy: 0.5884 - val_loss: 0.5572 - val_accuracy: 0.8664
Epoch 2/50
269/269 [==============================] - 210s 777ms/step - loss: 0.7090 - accuracy: 0.7684 - val_loss: 1.0661 - val_accuracy: 0.7563
Epoch 3/50
269/269 [==============================] - 215s 795ms/step - loss: 0.5608 - accuracy: 0.8183 - val_loss: 0.9638 - val_accuracy: 0.7707
Epoch 4/50
269/269 [==============================] - 215s 795ms/step - loss: 0.4683 - accuracy: 0.8413 - val_loss: 0.5267 - val_accuracy: 0.8808
Epoch 5/50
269/269 [==============================] - 214s 793ms/step - loss: 0.4129 - accuracy: 0.8596 - val_loss: 1.3406 - val_accuracy: 0.7739
Epoch 6/50
269/269 [==============================] - 213s 790ms/step - loss: 0.4200 - accuracy: 0.8618 - val_loss: 0.8258 - val_accuracy: 0.8602
Epoch 7/50
269/269 [==============================] - 214s 793ms/step - loss: 0.3485 - accuracy: 0.8825 - val_loss: 0.4853 -

In [ ]:
res_50_with_augment = pd.DataFrame(hist.history)
res_50_with_augment['time'] = time_callback.times
res_50_with_augment.to_csv('res_50_with_augment.csv')